In [4]:
pip install langchain-text-splitters

Note: you may need to restart the kernel to use updated packages.


In [6]:
import streamlit as st
import tempfile
import os

from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

# -------------------------------
# Streamlit Config
# -------------------------------
st.set_page_config(
    page_title="Student Lease Break Advisor",
    layout="wide"
)

st.title("Student Lease Break Advisor")
st.caption("Legal information assistant for students (not legal advice)")

# -------------------------------
# Utility Functions
# -------------------------------

def get_splitter():
    return RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=100,
        separators=["\n\n", "\n", "Section", "Clause"]
    )

def load_lease(file_path):
    ext = os.path.splitext(file_path)[1].lower()

    if ext == ".pdf":
        loader = PyPDFLoader(file_path)
    elif ext == ".docx":
        loader = Docx2txtLoader(file_path)
    else:
        raise ValueError("Unsupported file type")

    docs = loader.load()
    splitter = get_splitter()
    chunks = splitter.split_documents(docs)

    for d in chunks:
        d.metadata["source"] = "lease"

    return chunks

def load_laws(state):
    laws = {
        "New York": """
New York Real Property Law §227-e allows tenants to terminate a lease early under
certain conditions such as financial hardship, employment relocation, or medical
necessity, provided reasonable notice is given.

New York Real Property Law §235-b establishes the Warranty of Habitability, requiring
landlords to maintain safe and livable housing conditions. Failure to do so may
justify lease termination.
""",
        "California": """
California Civil Code §1946 permits lease termination with proper written notice
depending on lease type.

California Civil Code §1941 requires landlords to maintain habitable living
conditions. Failure to do so may justify lease termination.
"""
    }

    with tempfile.NamedTemporaryFile(delete=False, mode="w", suffix=".txt") as f:
        f.write(laws[state])
        law_path = f.name

    loader = TextLoader(law_path)
    docs = loader.load()

    splitter = get_splitter()
    chunks = splitter.split_documents(docs)

    for d in chunks:
        d.metadata["source"] = "law"
        d.metadata["state"] = state

    return chunks

def build_vectorstore(docs):
    embeddings = GoogleGenerativeAIEmbeddings(
        model="models/embedding-001"
    )
    return FAISS.from_documents(docs, embeddings)

def generate_answer(question, lease_db, law_db):
    lease_docs = lease_db.similarity_search(question, k=4)
    law_docs = law_db.similarity_search(question, k=4)

    context = "\n\n".join(
        [f"[{d.metadata['source'].upper()}] {d.page_content}"
         for d in lease_docs + law_docs]
    )

    llm = ChatGoogleGenerativeAI(
        model="gemini-2.5-pro",
        temperature=0
    )

    prompt = f"""
You are a legal information assistant for students.
You do NOT provide legal advice.
You must base your response strictly on the provided lease clauses and state laws.
If the information is insufficient, explicitly say so.

Question:
{question}

Context:
{context}

Respond with:
1. Whether early termination may be legally allowed
2. Required notice period (if specified)
3. Potential penalties or risks
4. Explicit citations (lease clause or statute)
5. A clear legal disclaimer
"""

    return llm.invoke(prompt).content

# -------------------------------
# Sidebar Inputs
# -------------------------------
state = st.sidebar.selectbox(
    "Select U.S. State",
    ["New York", "California"]
)

uploaded_file = st.sidebar.file_uploader(
    "Upload Lease (PDF or DOCX)",
    type=["pdf", "docx"]
)

st.sidebar.markdown(
    "**Disclaimer:** This tool provides legal information, not legal advice."
)

# -------------------------------
# Main Interaction
# -------------------------------
question = st.chat_input("Ask a question about breaking your lease")

if uploaded_file:
    with tempfile.NamedTemporaryFile(delete=False) as tmp:
        tmp.write(uploaded_file.read())
        lease_path = tmp.name

    with st.spinner("Processing lease document..."):
        lease_docs = load_lease(lease_path)
        lease_db = build_vectorstore(lease_docs)

    with st.spinner("Loading state tenant laws..."):
        law_docs = load_laws(state)
        law_db = build_vectorstore(law_docs)

    st.success("Lease and laws processed. You may ask questions.")

if question and uploaded_file:
    with st.spinner("Analyzing your question..."):
        response = generate_answer(question, lease_db, law_db)

    st.chat_message("assistant").write(response)


2026-01-08 13:27:04.147 
  command:

    streamlit run c:\Users\LOKESH\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
